# Use namedtuple as Row

In [1]:
from collections import namedtuple
import sqlite3

User = namedtuple('User', ['uid', 'alias', 'shell'])

In [2]:
conn = sqlite3.connect(':memory:')
conn.execute('CREATE TABLE user (uid INT, alias TEXT, shell TEXT)')
conn.commit()

In [3]:
# Insert a few rows
rows = [
    (501, 'john', 'bash'),
    (502, 'karen', 'tcsh'),
]
conn.executemany('INSERT INTO user VALUES (?, ?, ?)', rows)
conn.commit()

In [4]:
# Without row factory, each row is returned as a tuple
for row in conn.execute('SELECT * FROM user'):
    print(row)

(501, 'john', 'bash')
(502, 'karen', 'tcsh')


In [5]:
# With a row factory, a custom type is returned for each row
conn.row_factory = lambda cursor, row: User._make(row)
for row in conn.execute('SELECT * FROM user'):
    print(row)

User(uid=501, alias='john', shell='bash')
User(uid=502, alias='karen', shell='tcsh')
